<a href="https://colab.research.google.com/github/chengulatj/nominatim-reverse-geocoding/blob/main/Nominatim_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
locations = pd.read_excel('/content/drive/MyDrive/SL0202_Replacement_Locations.xlsx', skiprows=1)
locations.head()

,Roadway,Log Mile,IP,Location,County,Job Number,Switch,Expansion,Digital Logger,Latitude,Longitude
0,I-44,IS044W261.3C,10.6.126.170,I-44 @ Allenton/Six Flags,NaN,NaN,1,1.0,NaN,"38°30'22.99""N","90°40'40.82""W"
1,I-55,IS055N190.7Z,10.6.104.97,I-55 @ Richardson,NaN,NaN,1,1.0,NaN,"38°25'0.69""N","90°23'26.04""W"
2,I-55,IS055N198.0Z0,10.6.139.93,I-55 @ 61,NaN,NaN,1,1.0,1.0,"38°30'40.85""N","90°20'8.27""W"
3,I-55,IS055N199.1C,10.6.138.95,I-55 N of 61 (I-55 S of Reavis Barracks),NaN,NaN,1,1.0,NaN,"38°31'57.61""N","90°18'38.45""W"
4,I-55,IS055N202.7Z,10.6.100.236,I-55 @ Reavis Barracks East,NaN,NaN,1,1.0,NaN,"38°32'0.63""N","90°18'45.04""W"


In [13]:
#creating a sub-dataframe with the coordinates only
latlongs = locations[['Latitude', 'Longitude']]
latlongs.head()

,Latitude,Longitude
0,"38°30'22.99""N","90°40'40.82""W"
1,"38°25'0.69""N","90°23'26.04""W"
2,"38°30'40.85""N","90°20'8.27""W"
3,"38°31'57.61""N","90°18'38.45""W"
4,"38°32'0.63""N","90°18'45.04""W"


In [14]:
import re


# Define a helper function to convert Degree-Minute-Second (DMS) to decimal
def dms_to_dd(dms_str):
    # Convert the input to string to handle potential non-string values
    dms_str = str(dms_str)
    parts = re.split('[°\'"]+', dms_str)

    # Handle cases where the input is not in DMS format or is empty
    if len(parts) != 4:
        # the code will return NaN for invalid inputs
        return float('nan')

    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])
    direction = parts[3].strip()

    dd = degrees + minutes / 60 + seconds / 3600
    if direction in ['S', 'W']:
        dd *= -1  # South and West are negative
    return dd

# Applying the conversion to both Latitude and Longitude columns
# Use .loc[:, 'column_name'] to select the columns (this avoids warnings)
latlongs.loc[:,'Latitude_dec'] = latlongs.loc[:,'Latitude'].apply(dms_to_dd)
latlongs.loc[:,'Longitude_dec'] = latlongs.loc[:,'Longitude'].apply(dms_to_dd)

<ipython-input-14-c9250181fe04>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latlongs.loc[:,'Latitude_dec'] = latlongs.loc[:,'Latitude'].apply(dms_to_dd)
<ipython-input-14-c9250181fe04>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latlongs.loc[:,'Longitude_dec'] = latlongs.loc[:,'Longitude'].apply(dms_to_dd)


In [15]:
# Displaying the sample result
latlongs[['Latitude', 'Longitude', 'Latitude_dec', 'Longitude_dec']].head()

,Latitude,Longitude,Latitude_dec,Longitude_dec
0,"38°30'22.99""N","90°40'40.82""W",38.506386,-90.678006
1,"38°25'0.69""N","90°23'26.04""W",38.416858,-90.390567
2,"38°30'40.85""N","90°20'8.27""W",38.511347,-90.335631
3,"38°31'57.61""N","90°18'38.45""W",38.532669,-90.310681
4,"38°32'0.63""N","90°18'45.04""W",38.533508,-90.312511


In [16]:
!pip install geopy

In [17]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

# Initialize geopy's geolocator with a longer timeout and custom user-agent
geolocator = Nominatim(user_agent="my-geocoder-app", timeout=10)  # 10-second timeout


# Function to reverse geocode and get the county
def get_county(lat, lon):
    # Check if coordinates are valid before geocoding
    if pd.isna(lat) or pd.isna(lon):
        return 'Invalid Coordinates'
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location and 'county' in location.raw['address']:
            return location.raw['address']['county']
        return 'County not found'
    except (GeocoderTimedOut, GeocoderUnavailable):
        time.sleep(2)  # Wait for 2 seconds before retrying
        try:
            # Retry the request once
            location = geolocator.reverse((lat, lon), exactly_one=True)
            if location and 'county' in location.raw['address']:
                return location.raw['address']['county']
            return 'County not found'
        except (GeocoderTimedOut, GeocoderUnavailable):
            return 'Timed out'

In [18]:

# Reverse geocoding to find counties,  (using .loc will  avoid warnings)
latlongs.loc[:, 'County'] = latlongs.apply(lambda row: get_county(row['Latitude_dec'], row['Longitude_dec']), axis=1)

<ipython-input-18-daddf7ff86e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latlongs.loc[:, 'County'] = latlongs.apply(lambda row: get_county(row['Latitude_dec'], row['Longitude_dec']), axis=1)


In [19]:

# Display the result
latlongs[['Latitude', 'Longitude', 'Latitude_dec', 'Longitude_dec', 'County']].head()

,Latitude,Longitude,Latitude_dec,Longitude_dec,County
0,"38°30'22.99""N","90°40'40.82""W",38.506386,-90.678006,Saint Louis County
1,"38°25'0.69""N","90°23'26.04""W",38.416858,-90.390567,Jefferson County
2,"38°30'40.85""N","90°20'8.27""W",38.511347,-90.335631,Saint Louis County
3,"38°31'57.61""N","90°18'38.45""W",38.532669,-90.310681,Saint Louis County
4,"38°32'0.63""N","90°18'45.04""W",38.533508,-90.312511,Saint Louis County


In [20]:
#adding the newly generated column into the original dataset
locations['County'] = latlongs['County']
locations.head()

,Roadway,Log Mile,IP,Location,County,Job Number,Switch,Expansion,Digital Logger,Latitude,Longitude
0,I-44,IS044W261.3C,10.6.126.170,I-44 @ Allenton/Six Flags,Saint Louis County,NaN,1,1.0,NaN,"38°30'22.99""N","90°40'40.82""W"
1,I-55,IS055N190.7Z,10.6.104.97,I-55 @ Richardson,Jefferson County,NaN,1,1.0,NaN,"38°25'0.69""N","90°23'26.04""W"
2,I-55,IS055N198.0Z0,10.6.139.93,I-55 @ 61,Saint Louis County,NaN,1,1.0,1.0,"38°30'40.85""N","90°20'8.27""W"
3,I-55,IS055N199.1C,10.6.138.95,I-55 N of 61 (I-55 S of Reavis Barracks),Saint Louis County,NaN,1,1.0,NaN,"38°31'57.61""N","90°18'38.45""W"
4,I-55,IS055N202.7Z,10.6.100.236,I-55 @ Reavis Barracks East,Saint Louis County,NaN,1,1.0,NaN,"38°32'0.63""N","90°18'45.04""W"


In [21]:
locations.to_excel('/content/drive/MyDrive/SL0202_Replacement_Locations_with_County.xlsx')